# tools.py 디버깅 테스트

tool_turn_resolution 함수를 테스트합니다.

## 1. 환경 설정

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트를 path에 추가
root = Path.cwd()
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print(f"프로젝트 루트: {root}")

In [ ]:
import os
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG, format="%(levelname)s %(name)s %(message)s")

## 2. GPU / 로컬 모델 사용 여부 확인

In [ ]:
import torch

print("=" * 60)
print("GPU / 로컬 모델 확인")
print("=" * 60)

# 1. CUDA 가용성
print(f"\n[1] torch.cuda.is_available(): {torch.cuda.is_available()}")
print(f"    torch.version.cuda: {torch.version.cuda}")
print(f"    torch.__version__: {torch.__version__}")

# 2. GPU 정보
if torch.cuda.is_available():
    print(f"\n[2] GPU 정보:")
    print(f"    device_count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"    GPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"        총 메모리: {props.total_memory / 1024**3:.2f} GB")
    
    # 현재 GPU 메모리 사용량
    print(f"\n[3] GPU 메모리 사용량 (현재):")
    print(f"    allocated: {torch.cuda.memory_allocated() / 1024**3:.3f} GB")
    print(f"    reserved:  {torch.cuda.memory_reserved() / 1024**3:.3f} GB")
else:
    print("\n[!] CUDA를 사용할 수 없습니다. CPU 모드로 동작 중.")
    print("    GPU가 사용되지 않는 이유일 수 있습니다.")

# 3. nvidia-smi 정보 (subprocess)
print(f"\n[4] nvidia-smi 확인:")
import subprocess
try:
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=name,memory.used,memory.total,utilization.gpu", "--format=csv,noheader,nounits"],
        capture_output=True, text=True, timeout=5
    )
    if result.returncode == 0:
        for line in result.stdout.strip().split("\n"):
            parts = [p.strip() for p in line.split(",")]
            if len(parts) >= 4:
                name, mem_used, mem_total, util = parts
                print(f"    {name}: {mem_used}/{mem_total} MB, GPU 사용률: {util}%")
    else:
        print(f"    nvidia-smi 실행 실패: {result.stderr}")
except FileNotFoundError:
    print("    nvidia-smi를 찾을 수 없습니다.")
except Exception as e:
    print(f"    nvidia-smi 실행 오류: {e}")

## 2. 시나리오 로드

In [ ]:
from app.loader import ScenarioLoader

base_path = root / "scenarios"
loader = ScenarioLoader(base_path)
scenario_ids = loader.list_scenarios()

print(f"사용 가능한 시나리오: {scenario_ids}")

if not scenario_ids:
    raise Exception("시나리오가 없습니다!")

scenario_id = scenario_ids[0]
assets = loader.load(scenario_id)
print(f"\n로드된 시나리오: {scenario_id}")
print(f"  - 제목: {assets.scenario.get('title')}")
print(f"  - NPC: {assets.get_all_npc_ids()}")
print(f"  - 아이템: {assets.get_all_item_ids()}")

## 3. 월드 상태 생성

In [ ]:
from app.schemas import WorldState, NPCState

world = WorldState(
    turn=1,
    npcs={
        "family": NPCState(npc_id="family", trust=0, fear=0, suspicion=0),
        "partner": NPCState(npc_id="partner", trust=0, fear=0, suspicion=1),
        "witness": NPCState(npc_id="witness", trust=0, fear=2, suspicion=0),
    },
    inventory=["casefile_brief", "pattern_analyzer"],
    vars={"clue_count": 0, "fabrication_score": 0},
)

print(f"WorldState:")
print(f"  - turn: {world.turn}")
print(f"  - npcs: {list(world.npcs.keys())}")
print(f"  - inventory: {world.inventory}")
print(f"  - vars: {world.vars}")

## 4. LLM 로드

In [ ]:
from app.llm import LLM_Engine

model_name = os.environ.get("LLM_MODEL", "Qwen/Qwen3-8B")
print(f"LLM 로딩 중: {model_name}")
print("(환경변수 LLM_MODEL로 변경 가능)")

# GPU 메모리 로드 전 측정
if torch.cuda.is_available():
    torch.cuda.synchronize()
    mem_before = torch.cuda.memory_allocated()
    print(f"\n[모델 로드 전] GPU 메모리: {mem_before / 1024**3:.3f} GB")

llm = LLM_Engine(model_name=model_name)
print("\nLLM 로드 완료!")

# GPU 메모리 로드 후 측정
if torch.cuda.is_available():
    torch.cuda.synchronize()
    mem_after = torch.cuda.memory_allocated()
    print(f"[모델 로드 후] GPU 메모리: {mem_after / 1024**3:.3f} GB")
    print(f"[모델 크기] {(mem_after - mem_before) / 1024**3:.3f} GB")

### 4-1. 모델 디바이스 위치 및 상세 정보

In [ ]:
# LLM_Engine이 사용하는 디바이스 확인
print("=" * 60)
print("모델 디바이스 및 상세 정보")
print("=" * 60)

print(f"\n[1] LLM_Engine.device: {llm.device}")

# 모델 파라미터가 어디에 있는지 확인
print(f"\n[2] 모델 파라미터 위치:")
try:
    # 첫 번째 파라미터의 디바이스 확인
    first_param = next(llm.model.parameters())
    print(f"    첫 번째 파라미터 device: {first_param.device}")
    print(f"    dtype: {first_param.dtype}")
    
    # 모든 파라미터가 같은 디바이스에 있는지 확인
    devices = set()
    for name, param in llm.model.named_parameters():
        devices.add(str(param.device))
    print(f"    사용 중인 디바이스들: {devices}")
    
    if len(devices) == 1 and "cuda" in list(devices)[0]:
        print("\n    ✓ 모델이 GPU에 로드되어 있습니다.")
    elif len(devices) == 1 and "cpu" in list(devices)[0]:
        print("\n    ✗ 모델이 CPU에 로드되어 있습니다!")
        print("      → GPU가 사용되지 않는 원인입니다.")
    else:
        print(f"\n    모델이 여러 디바이스에 분산되어 있습니다: {devices}")
except Exception as e:
    print(f"    파라미터 확인 실패: {e}")

# 모델 config 정보
print(f"\n[3] 모델 정보:")
print(f"    model_type: {getattr(llm.model.config, 'model_type', 'unknown')}")
print(f"    hidden_size: {getattr(llm.model.config, 'hidden_size', 'unknown')}")
print(f"    num_layers: {getattr(llm.model.config, 'num_hidden_layers', 'unknown')}")

# 파라미터 수 계산
total_params = sum(p.numel() for p in llm.model.parameters())
print(f"    총 파라미터 수: {total_params:,} ({total_params / 1e9:.2f}B)")

## 5. 프롬프트 확인

In [ ]:
from app.llm import build_prompt

user_input = "피해자 가족에게 그날 있었던 일을 물어본다"

prompt = build_prompt(
    user_input=user_input,
    world_state=world.to_dict(),
    memory_summary=None,
    npc_context=assets.export_for_prompt(),
)

print(f"프롬프트 길이: {len(prompt)} 자")
print("=" * 60)
print(prompt)
print("=" * 60)

## 6. tool_turn_resolution 테스트

In [ ]:
import time
import threading

def monitor_gpu_during_inference():
    """Inference 중 GPU 사용률 모니터링 (별도 스레드)"""
    max_util = 0
    max_mem = 0
    samples = []
    
    def sample():
        nonlocal max_util, max_mem
        while getattr(monitor_gpu_during_inference, 'running', True):
            try:
                result = subprocess.run(
                    ["nvidia-smi", "--query-gpu=utilization.gpu,memory.used", "--format=csv,noheader,nounits"],
                    capture_output=True, text=True, timeout=1
                )
                if result.returncode == 0:
                    parts = [p.strip() for p in result.stdout.strip().split(",")]
                    if len(parts) >= 2:
                        util = int(parts[0])
                        mem = int(parts[1])
                        max_util = max(max_util, util)
                        max_mem = max(max_mem, mem)
                        samples.append((util, mem))
            except:
                pass
            time.sleep(0.1)  # 100ms 간격 샘플링
    
    return sample, lambda: (max_util, max_mem, samples)

print("=" * 60)
print("Inference 중 GPU 사용률 모니터링")
print("=" * 60)

# GPU 모니터링 시작
sampler, get_stats = monitor_gpu_during_inference()
monitor_gpu_during_inference.running = True
monitor_thread = threading.Thread(target=sampler, daemon=True)
monitor_thread.start()

# inference 전 상태
if torch.cuda.is_available():
    torch.cuda.synchronize()
    mem_before_inference = torch.cuda.memory_allocated()
    torch.cuda.reset_peak_memory_stats()

print(f"\n[Inference 전] GPU 메모리: {torch.cuda.memory_allocated() / 1024**3:.3f} GB")

# inference 실행
start_time = time.perf_counter()
test_prompt = "안녕하세요"
print(f"테스트 프롬프트: '{test_prompt}'")
print("Inference 실행 중...")

test_output = llm.generate(test_prompt)

end_time = time.perf_counter()
inference_time = end_time - start_time

# 모니터링 중지
monitor_gpu_during_inference.running = False
time.sleep(0.2)  # 마지막 샘플 수집 대기

# 결과 수집
max_util, max_mem, samples = get_stats()

# inference 후 상태
if torch.cuda.is_available():
    torch.cuda.synchronize()
    mem_after_inference = torch.cuda.memory_allocated()
    peak_mem = torch.cuda.max_memory_allocated()

print(f"\n[결과]")
print(f"    Inference 시간: {inference_time:.2f}초")
print(f"    출력 길이: {len(test_output)}자")

if torch.cuda.is_available():
    print(f"\n[GPU 메모리]")
    print(f"    inference 전: {mem_before_inference / 1024**3:.3f} GB")
    print(f"    inference 후: {mem_after_inference / 1024**3:.3f} GB")
    print(f"    peak: {peak_mem / 1024**3:.3f} GB")

print(f"\n[nvidia-smi 모니터링 결과]")
print(f"    샘플 수: {len(samples)}")
if samples:
    avg_util = sum(s[0] for s in samples) / len(samples)
    avg_mem = sum(s[1] for s in samples) / len(samples)
    print(f"    평균 GPU 사용률: {avg_util:.1f}%")
    print(f"    최대 GPU 사용률: {max_util}%")
    print(f"    평균 GPU 메모리: {avg_mem:.0f} MB")
    print(f"    최대 GPU 메모리: {max_mem} MB")
    
    if max_util < 5:
        print("\n    ⚠ GPU 사용률이 매우 낮습니다 (< 5%)!")
        print("      → 로컬 모델이 실제로 GPU에서 실행되지 않을 가능성이 있습니다.")
    elif max_util < 30:
        print("\n    ⚠ GPU 사용률이 낮습니다 (< 30%).")
        print("      → 모델이 GPU에 있지만, 병목이 다른 곳에 있을 수 있습니다.")
    else:
        print("\n    ✓ GPU가 활발하게 사용되고 있습니다.")
else:
    print("    샘플을 수집하지 못했습니다.")

print(f"\n[생성된 텍스트 (처음 200자)]")
print("-" * 40)
print(test_output[:200] if len(test_output) > 200 else test_output)
print("-" * 40)

In [ ]:
from app.tools import tool_turn_resolution

user_input = "피해자 가족에게 그날 있었던 일을 물어본다"

print(f"입력: {user_input}")
print("LLM 생성 중...")

result = tool_turn_resolution(user_input, world, assets, llm)

print("\n결과:")
print(f"  event_description: {result.event_description}")
print(f"  state_delta: {result.state_delta}")

## 7. 여러 테스트 케이스

In [ ]:
test_cases = [
    "피해자 가족에게 그날 있었던 일을 물어본다",
    "패턴 분석기를 사용한다",
    "현장을 조사한다",
]

print(f"테스트 케이스 ({len(test_cases)}개)")
print("=" * 60)

for i, text in enumerate(test_cases, 1):
    print(f"\n[{i}] 입력: \"{text}\"")
    result = tool_turn_resolution(text, world, assets, llm)
    print(f"    사건: {result.event_description}")
    print(f"    델타: {result.state_delta}")

print("\n" + "=" * 60)
print("테스트 완료!")

## 8. Raw LLM 출력 확인

In [ ]:
from app.llm import parse_response

user_input = "목격자에게 범인의 인상착의를 물어본다"

prompt = build_prompt(
    user_input=user_input,
    world_state=world.to_dict(),
    memory_summary=None,
    npc_context=assets.export_for_prompt(),
)

print(f"입력: {user_input}")
print("\nLLM 생성 중...")

raw_output = llm.generate(prompt)

print("\n[Raw LLM Output]")
print("-" * 40)
print(raw_output)
print("-" * 40)

print("\n[Parsed Response]")
parsed = parse_response(raw_output)
print(f"  state_delta: {parsed.state_delta}")
print(f"  event_description: {parsed.event_description}")

## 9. 로컬 모델 vs API 확인 요약

In [ ]:
print("=" * 60)
print("진단 요약")
print("=" * 60)

issues = []
info = []

# 1. CUDA 가용성 확인
if not torch.cuda.is_available():
    issues.append("CUDA를 사용할 수 없음 - CPU 모드로 동작 중")
else:
    info.append(f"CUDA 사용 가능: {torch.cuda.get_device_name(0)}")

# 2. 모델 디바이스 확인
try:
    first_param = next(llm.model.parameters())
    if "cpu" in str(first_param.device):
        issues.append("모델이 CPU에 로드되어 있음")
    else:
        info.append(f"모델 디바이스: {first_param.device}")
except:
    issues.append("모델 파라미터 확인 실패")

# 3. GPU 메모리 확인
if torch.cuda.is_available():
    gpu_mem = torch.cuda.memory_allocated() / 1024**3
    if gpu_mem < 0.1:
        issues.append(f"GPU 메모리 사용량이 매우 낮음 ({gpu_mem:.3f} GB)")
    else:
        info.append(f"GPU 메모리 사용: {gpu_mem:.2f} GB")

# 4. LangChain 엔진 사용 여부 확인
print("\n[LLM 엔진 타입 확인]")
print(f"    사용 중인 엔진: {type(llm).__name__}")
if type(llm).__name__ == "LLM_Engine":
    info.append("LLM_Engine (로컬 transformers) 사용 중")
elif "LangChain" in type(llm).__name__:
    issues.append("LangChainEngine (API) 사용 중 - 로컬 GPU 미사용!")

# 결과 출력
print("\n" + "-" * 60)
if issues:
    print("\n⚠ 발견된 문제점:")
    for issue in issues:
        print(f"    - {issue}")
else:
    print("\n✓ 문제점 없음")

print("\n📋 시스템 정보:")
for item in info:
    print(f"    - {item}")

# 최종 판정
print("\n" + "-" * 60)
print("\n[최종 판정]")

if not torch.cuda.is_available():
    print("    ❌ 로컬 GPU 미사용: CUDA가 사용 불가능합니다.")
    print("       → PyTorch CUDA 버전 재설치 필요")
elif "cpu" in str(next(llm.model.parameters()).device):
    print("    ❌ 로컬 GPU 미사용: 모델이 CPU에 로드되어 있습니다.")
    print("       → LLM_Engine의 device 설정 확인 필요")
elif torch.cuda.memory_allocated() < 1024**3:  # 1GB 미만
    print("    ⚠ GPU 메모리 사용량이 낮습니다.")
    print("       → 모델이 제대로 로드되지 않았거나, 다른 문제가 있을 수 있습니다.")
else:
    print("    ✓ 로컬 GPU 사용 중: 모델이 GPU에서 실행되고 있습니다.")
    print("       → GPU 사용률이 낮다면, inference 시간이 짧거나 I/O 병목일 수 있습니다.")

print("\n" + "=" * 60)